In [7]:
import subprocess
import os
import datetime
import logging

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [8]:
def mail_script_failed(script_name):
    html_head = """
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40">
    <head><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=us-ascii"><meta name=Generator content="Microsoft Word 15 (filtered medium)">
    </head>"""
    
    toaddr = ['jworms@nova401k.com', 'msvehla@nova401k.com']
    cc_list = []

    html = html_head

    fromaddr = 'automation@nova401k.com'
    password = 'Rub73595'

    toaddrs = toaddr + cc_list

    if len(toaddrs) == 0:
        x = "No emails to send"
    else:
        msg = MIMEMultipart('alternative')

        msg['From'] = fromaddr
        msg['To'] = ','.join(toaddr)
        msg['CC'] = ','.join(cc_list)
        subject = f'Script {script_name} failed to run today'
        msg['Subject'] = subject
        part = MIMEText(html, 'html')
        msg.attach(part)
        


        with smtplib.SMTP('smtp.office365.com', 587) as server:
            server.starttls()
            server.login(fromaddr, password)
            x = server.sendmail(fromaddr, toaddrs, msg.as_string())


In [9]:
def run_child_script(script_name, log_folder):
    try:
        # Create log folder if it doesn't exist
        os.makedirs(log_folder, exist_ok=True)
        
        # Define log file path
        log_file = os.path.join(log_folder, f"{script_name}_log.txt")

        # Open log file in append mode
        with open(log_file, "a") as f:
            # Run the child script and capture stdout and stderr
            subprocess.run(["python", script_name], stdout=f, stderr=subprocess.STDOUT, check=True)
    except Exception as e:
        print(f"Error running {script_name}: {e}")
        raise
#         mail_script_failed(script_name)       


In [11]:
def main():

    child_scripts = ["Form 8955-SSA Project Launch Triggers (Ticket #25009).py", 
                     "Form 8955-SSA (Automated) task Completion - Form 8955-SSA (Ticket #25012) Part 1.py",
                     "Form 8955-SSA (Automated) task Completion - Form 8955-SSA  (Ticket #25012) Part 2.py",
                     "Form 8955-SSA Project - Delivery- Form 8955-SSA Task Automation Steps (Ticket #25039).py"]
    
    dt = datetime.datetime.now().strftime('%Y.%m.%d.%I.%M.%S')
    log_folder = r"Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\Logs" 
    combined_log_folder = r"Y:\Automation\Scheduled\stdout" 
    combined_log_file = f"{combined_log_folder}\Form_8955SSA_combined_log_{dt}.txt"
    log_file = os.path.join(log_folder, "parent_script.log")
    
    logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )

    logger = logging.getLogger(__name__)
    logger.info(f'\n\n\n------------------ {dt} --------------')
    logger.info('Parent!!!!!!!!!!!!!!!')
    logger.info("Starting parent script")
    
#     Run each child script sequentially
    for script_name in child_scripts:
        logger.info(f"Running {script_name}...")
        try:
            run_child_script(script_name, log_folder)
        except Exception as e:
            print("Halting further execution due to error in child script.")
            logger.info("Halting further execution due to error in child script.")
            break
        logger.info(f"Completed {script_name}\n")
        
    logger.info(f"Combined log file written to {combined_log_file}")
    logger.info("Parent script completed")

In [12]:
if __name__ == "__main__":
    main()


Error combining log files: [WinError 32] The process cannot access the file because it is being used by another process: 'Y:\\Automation\\Team Scripts\\Anjana Shaji\\8955 SSA DF\\Logs\\Form 8955-SSA (Automated) task Completion - Form 8955-SSA  (Ticket #25012) Part 2.py_log.txt'
